1. найти число событий в конкретный день

Это при условии, что файл в той же директории, что и код

In [102]:
import pandas as pd
from datetime import datetime, timedelta

def count_sessions():
    '''Функция подсчета количества сессий'''
    # Открыть файл для чтения
    with open('log.csv', 'r') as f:
        # Чтение файла формата csv
        df = pd.read_csv(f)
        # Преобразование времени в формат datetime
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d_%H:%M:%S')
        # Сортировка по пользователю и времени
        df = df.sort_values(['user', 'date'])
        # Вычисление разницы между событиями пользователя
        df['time_diff'] = df.groupby('user')['date'].diff()
        # Определяем начало новых сессий (первый эвент пользователя или разница >= 30 минут)
        df['new_session'] = (df['time_diff'].isna()) | (df['time_diff'] >= timedelta(minutes=30))
        # Считаем сессии, начавшиеся 2020-04-19
        target_date = pd.to_datetime('2020-04-19').date()
        # Условие на новую сессию пользователя в нужную дату
        sessions_count = df[df['new_session'] & (df['date'].dt.date == target_date)].shape[0]
        return sessions_count, df

In [103]:
print(count_sessions()[0])

6251


Для ссылки надо будет написать через requests. Поскольку прямая ссылка на Яндекс.Диск ведет на HTML-страницу с капчей, то чтобы\
получить сам файл, нужно использовать API Яндекс.Диска

In [104]:
import requests
from io import StringIO

def count_sessions_url():
    '''Функиця для подсчета количества сессий по ссылке на файл Яндекс.Диска'''
    # Ссылка. Увидеть ее можно, перейдя по ссылке в README.md, появится https://disk.yandex.ru/d/nGRwE9x0n9YEY
    # Префикс перед ней - как раз и есть работа с API
    url = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/nGRwE9x0n9YEYA'
    response = requests.get(url)
    data = pd.read_csv(StringIO(response.text))
    # На этом содержательное измнение заканчивается
    # Преобразование времени в формат datetime
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d_%H:%M:%S')
    # Сортировка по пользователю и времени
    data = data.sort_values(['user', 'date'])
    # Вычисление разницы между событиями пользователя
    data['time_diff'] = data.groupby('user')['date'].diff()
    # Определяем начало новых сессий (первый эвент пользователя или разница >= 30 минут)
    data['new_session'] = (data['time_diff'].isna()) | (data['time_diff'] >= timedelta(minutes=30))
    # Считаем сессии, начавшиеся 2020-04-19
    target_date = pd.to_datetime('2020-04-19').date()
    # Условие на новую сессию пользователя в нужную дату
    sessions_count = data[data['new_session'] & (data['date'].dt.date == target_date)].shape[0]
    return sessions_count, data

In [ ]:
print(count_sessions_url()[0])

Тут подробнее про API [клац](https://github.com/resources/articles/software-development/what-is-an-api?ysclid=md1p12crm4542903030)

На глаз можно ошибиться в цифрах, поэтому пусть решит компилятор, одинаковый ответ или нет

In [ ]:
print(count_sessions_url()[0] ==  count_sessions()[0])

2. найти день, когда больше всего пользователей смотрели видео

In [ ]:
# Присвоение прочитанного файла в результате работы первой функции
data = count_sessions()[1]
# Условие на событие = 2 или параметр = video, удаляет дубликаты в столбцах с датой сессии и пользователем
video = data[(data['event_type'] == 2) & (data['parameter'] == 'video')]
video['day'] = video['date'].dt.date
# Группировка по дате, поиск уникального значения в столбце пользователей, вывод максимального количества уникальных пользователей
print('Максимум:\t', video.groupby('day')['user'].nunique().max())

3. найти 5-и минутный интервал, в котором произошло больше всего событий

In [ ]:
# Преобразование дат в список по возрастанию часа
times = data['date'].sort_values().tolist()
# Максимальное количество и значение левой части интервала вначале присваивается нулю
max_count = left = 0
# Переменная содержит начало наиболее активного 5-и минутного интервала
best_start = None
# Right - индекс текущего интервала, time - текущее значение даты/времени
for right, time in enumerate(times):
    # В рассматриваемом интервале [left, right] все временные метки укладываются в 5 минут
    while time - times[left] >= timedelta(minutes=5):
        # Сдвигает левую границу (left), пока разница между текущим временем (time) и временем на 
        # левой границе (times[left]) не станет меньше 5 минут
        left += 1
    # Вычисляет количество элементов в текущем интервале (count = right - left + 1)
    if (count := right - left + 1) >= max_count:
        # Если это количество больше или равно текущему максимуму, обновляет max_count 
        # и запоминает начало интервала (best_start = times[left])
        max_count, best_start = count, times[left]
best_start.strftime('%Y-%m-%d_%H:%M:%S')